<a href="https://colab.research.google.com/github/shivammehta007/NLPResearch/blob/master/Tutorials/Natural%20Language%20Processing/PyTorch%20Sentimental%20Analysis/BiDirectional_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import torch.nn.functional as F
import random

In [2]:
SEED = 1234
torch.manual_seed(SEED)

In [0]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [5]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:09<00:00, 8.81MB/s]


In [0]:
traindata, validdata = trainset.split(random_state=random.seed(SEED))

In [9]:
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(traindata, max_size=MAX_VOCAB_SIZE, vectors= 'glove.6B.100d', unk_init=torch.Tensor.normal_)
LABEL.build_vocab(traindata)

.vector_cache/glove.6B.zip: 862MB [00:53, 16.1MB/s]                           
100%|█████████▉| 399717/400000 [00:16<00:00, 24268.44it/s]

In [13]:
torch.cuda.is_available()

True

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
BATCH_SIZE = 64

# Generate Iterators 

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (traindata, validdata, testset),
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=device)